In [1]:
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

In [2]:
train = pd.read_csv("train_v9rqX0R.csv")

In [3]:
test = pd.read_csv("test_AbJTz2l.csv")

In [4]:
sample_submission = pd.read_csv("sample_submission_8RXa3c6.csv")

In [5]:
train.head(10)

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.300,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.920,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.500,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.200,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.930,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052
5,FDP36,10.395,Regular,0.000000,Baking Goods,51.4008,OUT018,2009,Medium,Tier 3,Supermarket Type2,556.6088
6,FDO10,13.650,Regular,0.012741,Snack Foods,57.6588,OUT013,1987,High,Tier 3,Supermarket Type1,343.5528
7,FDP10,NaN,Low Fat,0.127470,Snack Foods,107.7622,OUT027,1985,Medium,Tier 3,Supermarket Type3,4022.7636
8,FDH17,16.200,Regular,0.016687,Frozen Foods,96.9726,OUT045,2002,NaN,Tier 2,Supermarket Type1,1076.5986
9,FDU28,19.200,Regular,0.094450,Frozen Foods,187.8214,OUT017,2007,NaN,Tier 2,Supermarket Type1,4710.5350


In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [7]:
train.describe(include = 'all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Item_Identifier,8523,1559,FDW13,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Item_Weight,7060.0,NaN,NaN,NaN,12.857645,4.643456,4.555,8.77375,12.6,16.85,21.35
Item_Fat_Content,8523,5,Low Fat,5089,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Item_Visibility,8523.0,NaN,NaN,NaN,0.066132,0.051598,0.0,0.026989,0.053931,0.094585,0.328391
Item_Type,8523,16,Fruits and Vegetables,1232,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Item_MRP,8523.0,NaN,NaN,NaN,140.992782,62.275067,31.29,93.8265,143.0128,185.6437,266.8884
Outlet_Identifier,8523,10,OUT027,935,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Outlet_Establishment_Year,8523.0,NaN,NaN,NaN,1997.831867,8.37176,1985.0,1987.0,1999.0,2004.0,2009.0
Outlet_Size,6113,3,Medium,2793,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Outlet_Location_Type,8523,3,Tier 3,3350,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Problems
# Item_Identifier = object
# Item_Weight = null value
# Item_Fat_Content = object
# Item_Visibility = 0 values
# Item_Type = object
# Item_MRP = 
# Outlet_Identifier = object
# Outlet_Establishment_Year = convert to age
# Outlet_Size = object
# Outlet_Location_Type = object
# Outlet_Type = object
# Item_Outlet_Sales =

In [9]:
train['source'] = 'train'
test['source'] = 'test'

In [10]:
data = pd.concat([train, test])

In [11]:
data.shape

(14204, 13)

In [12]:
data.head(10)

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,source
0,FDA15,9.300,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,train
1,DRC01,5.920,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,train
2,FDN15,17.500,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700,train
3,FDX07,19.200,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800,train
4,NCD19,8.930,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,train
5,FDP36,10.395,Regular,0.000000,Baking Goods,51.4008,OUT018,2009,Medium,Tier 3,Supermarket Type2,556.6088,train
6,FDO10,13.650,Regular,0.012741,Snack Foods,57.6588,OUT013,1987,High,Tier 3,Supermarket Type1,343.5528,train
7,FDP10,NaN,Low Fat,0.127470,Snack Foods,107.7622,OUT027,1985,Medium,Tier 3,Supermarket Type3,4022.7636,train
8,FDH17,16.200,Regular,0.016687,Frozen Foods,96.9726,OUT045,2002,NaN,Tier 2,Supermarket Type1,1076.5986,train
9,FDU28,19.200,Regular,0.094450,Frozen Foods,187.8214,OUT017,2007,NaN,Tier 2,Supermarket Type1,4710.5350,train


In [13]:
data.isna().sum()

Item_Identifier                 0
Item_Weight                  2439
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  4016
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales            5681
source                          0
dtype: int64

In [14]:
data['Item_Weight'] = data.groupby(['Item_Identifier', 'Item_Type']).Item_Weight.transform(lambda x: x.fillna(x.mean()))

In [15]:
data.head(10)

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,source
0,FDA15,9.300,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,train
1,DRC01,5.920,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,train
2,FDN15,17.500,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700,train
3,FDX07,19.200,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800,train
4,NCD19,8.930,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,train
5,FDP36,10.395,Regular,0.000000,Baking Goods,51.4008,OUT018,2009,Medium,Tier 3,Supermarket Type2,556.6088,train
6,FDO10,13.650,Regular,0.012741,Snack Foods,57.6588,OUT013,1987,High,Tier 3,Supermarket Type1,343.5528,train
7,FDP10,19.000,Low Fat,0.127470,Snack Foods,107.7622,OUT027,1985,Medium,Tier 3,Supermarket Type3,4022.7636,train
8,FDH17,16.200,Regular,0.016687,Frozen Foods,96.9726,OUT045,2002,NaN,Tier 2,Supermarket Type1,1076.5986,train
9,FDU28,19.200,Regular,0.094450,Frozen Foods,187.8214,OUT017,2007,NaN,Tier 2,Supermarket Type1,4710.5350,train


In [16]:
data.isna().sum()

Item_Identifier                 0
Item_Weight                     0
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  4016
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales            5681
source                          0
dtype: int64

In [17]:
data['Outlet_Size'] = data.groupby(['Outlet_Type']).Outlet_Size.transform(lambda x: x.fillna(x.mode()[0]))

In [18]:
data.head(10)

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,source
0,FDA15,9.300,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,train
1,DRC01,5.920,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,train
2,FDN15,17.500,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700,train
3,FDX07,19.200,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,Small,Tier 3,Grocery Store,732.3800,train
4,NCD19,8.930,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,train
5,FDP36,10.395,Regular,0.000000,Baking Goods,51.4008,OUT018,2009,Medium,Tier 3,Supermarket Type2,556.6088,train
6,FDO10,13.650,Regular,0.012741,Snack Foods,57.6588,OUT013,1987,High,Tier 3,Supermarket Type1,343.5528,train
7,FDP10,19.000,Low Fat,0.127470,Snack Foods,107.7622,OUT027,1985,Medium,Tier 3,Supermarket Type3,4022.7636,train
8,FDH17,16.200,Regular,0.016687,Frozen Foods,96.9726,OUT045,2002,Small,Tier 2,Supermarket Type1,1076.5986,train
9,FDU28,19.200,Regular,0.094450,Frozen Foods,187.8214,OUT017,2007,Small,Tier 2,Supermarket Type1,4710.5350,train


In [19]:
data.isna().sum()

Item_Identifier                 0
Item_Weight                     0
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                     0
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales            5681
source                          0
dtype: int64

In [20]:
data['Outlet_Establishment_Year'] = 2022 - data['Outlet_Establishment_Year']

In [21]:
data.head(10)

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,source
0,FDA15,9.300,Low Fat,0.016047,Dairy,249.8092,OUT049,23,Medium,Tier 1,Supermarket Type1,3735.1380,train
1,DRC01,5.920,Regular,0.019278,Soft Drinks,48.2692,OUT018,13,Medium,Tier 3,Supermarket Type2,443.4228,train
2,FDN15,17.500,Low Fat,0.016760,Meat,141.6180,OUT049,23,Medium,Tier 1,Supermarket Type1,2097.2700,train
3,FDX07,19.200,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,24,Small,Tier 3,Grocery Store,732.3800,train
4,NCD19,8.930,Low Fat,0.000000,Household,53.8614,OUT013,35,High,Tier 3,Supermarket Type1,994.7052,train
5,FDP36,10.395,Regular,0.000000,Baking Goods,51.4008,OUT018,13,Medium,Tier 3,Supermarket Type2,556.6088,train
6,FDO10,13.650,Regular,0.012741,Snack Foods,57.6588,OUT013,35,High,Tier 3,Supermarket Type1,343.5528,train
7,FDP10,19.000,Low Fat,0.127470,Snack Foods,107.7622,OUT027,37,Medium,Tier 3,Supermarket Type3,4022.7636,train
8,FDH17,16.200,Regular,0.016687,Frozen Foods,96.9726,OUT045,20,Small,Tier 2,Supermarket Type1,1076.5986,train
9,FDU28,19.200,Regular,0.094450,Frozen Foods,187.8214,OUT017,15,Small,Tier 2,Supermarket Type1,4710.5350,train


In [22]:
data_numbers = data.select_dtypes(include = np.number)

In [23]:
scale = StandardScaler()
scaled = scale.fit_transform(data_numbers.drop(columns = 'Item_Outlet_Sales'))
data_numbers = pd.DataFrame(scaled, columns = data_numbers.drop(columns = 'Item_Outlet_Sales').columns)


In [24]:
data_numbers

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year
0,-0.751014,-0.969852,1.752511,-0.139681
1,-1.477653,-0.907063,-1.493696,-1.334228
2,1.011839,-0.956000,0.009874,-0.139681
3,1.377308,-1.281712,0.661838,-0.020226
4,-0.830557,-1.281712,-1.403623,1.293777
...,...,...,...,...
14199,-0.493036,-1.019425,0.005000,0.099229
14200,-1.116483,1.497142,0.453249,-1.334228
14201,-0.600527,0.147226,-0.358558,-0.498045
14202,0.538878,-1.281712,1.185747,-1.095319


In [25]:
data['Item_Identifier'].value_counts()

FDU15    10
FDS25    10
FDA38    10
FDW03    10
FDJ10    10
         ..
FDR51     7
FDM52     7
DRN11     7
FDH58     7
NCW54     7
Name: Item_Identifier, Length: 1559, dtype: int64

In [26]:
data['Item_Identifier'].str[0:2].value_counts()

FD    10201
NC     2686
DR     1317
Name: Item_Identifier, dtype: int64

In [27]:
data['Item_Identifier_code'] = data['Item_Identifier'].str[0:2]

In [28]:
data.head(10)

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,source,Item_Identifier_code
0,FDA15,9.300,Low Fat,0.016047,Dairy,249.8092,OUT049,23,Medium,Tier 1,Supermarket Type1,3735.1380,train,FD
1,DRC01,5.920,Regular,0.019278,Soft Drinks,48.2692,OUT018,13,Medium,Tier 3,Supermarket Type2,443.4228,train,DR
2,FDN15,17.500,Low Fat,0.016760,Meat,141.6180,OUT049,23,Medium,Tier 1,Supermarket Type1,2097.2700,train,FD
3,FDX07,19.200,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,24,Small,Tier 3,Grocery Store,732.3800,train,FD
4,NCD19,8.930,Low Fat,0.000000,Household,53.8614,OUT013,35,High,Tier 3,Supermarket Type1,994.7052,train,NC
5,FDP36,10.395,Regular,0.000000,Baking Goods,51.4008,OUT018,13,Medium,Tier 3,Supermarket Type2,556.6088,train,FD
6,FDO10,13.650,Regular,0.012741,Snack Foods,57.6588,OUT013,35,High,Tier 3,Supermarket Type1,343.5528,train,FD
7,FDP10,19.000,Low Fat,0.127470,Snack Foods,107.7622,OUT027,37,Medium,Tier 3,Supermarket Type3,4022.7636,train,FD
8,FDH17,16.200,Regular,0.016687,Frozen Foods,96.9726,OUT045,20,Small,Tier 2,Supermarket Type1,1076.5986,train,FD
9,FDU28,19.200,Regular,0.094450,Frozen Foods,187.8214,OUT017,15,Small,Tier 2,Supermarket Type1,4710.5350,train,FD


In [29]:
data_hot = data.select_dtypes(exclude = np.number)

In [30]:
data_hot_encode = pd.get_dummies(data_hot.drop(columns = 'Item_Identifier'))

In [31]:
data_hot_encode.head(10)

,Item_Fat_Content_LF,Item_Fat_Content_Low Fat,Item_Fat_Content_Regular,Item_Fat_Content_low fat,Item_Fat_Content_reg,Item_Type_Baking Goods,Item_Type_Breads,Item_Type_Breakfast,Item_Type_Canned,Item_Type_Dairy,...,Outlet_Location_Type_Tier 3,Outlet_Type_Grocery Store,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3,source_test,source_train,Item_Identifier_code_DR,Item_Identifier_code_FD,Item_Identifier_code_NC
0,0,1,0,0,0,0,0,0,0,1,...,0,0,1,0,0,0,1,0,1,0
1,0,0,1,0,0,0,0,0,0,0,...,1,0,0,1,0,0,1,1,0,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,1,0
3,0,0,1,0,0,0,0,0,0,0,...,1,1,0,0,0,0,1,0,1,0
4,0,1,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,1,0,0,1
5,0,0,1,0,0,1,0,0,0,0,...,1,0,0,1,0,0,1,0,1,0
6,0,0,1,0,0,0,0,0,0,0,...,1,0,1,0,0,0,1,0,1,0
7,0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,1,0,1,0
8,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,1,0
9,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,1,0


In [32]:
data_numbers = data_numbers.reset_index(drop=True)
data_hot_encode = data_hot_encode.reset_index(drop=True )
data = data.reset_index(drop =True)

In [33]:
data_preprocessed = pd.concat([data_numbers, data_hot_encode, data[['Item_Identifier','Outlet_Identifier','source','Item_Outlet_Sales']]], axis =1)

In [34]:
data_preprocessed.head(10)

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Fat_Content_LF,Item_Fat_Content_Low Fat,Item_Fat_Content_Regular,Item_Fat_Content_low fat,Item_Fat_Content_reg,Item_Type_Baking Goods,...,Outlet_Type_Supermarket Type3,source_test,source_train,Item_Identifier_code_DR,Item_Identifier_code_FD,Item_Identifier_code_NC,Item_Identifier,Outlet_Identifier,source,Item_Outlet_Sales
0,-0.751014,-0.969852,1.752511,-0.139681,0,1,0,0,0,0,...,0,0,1,0,1,0,FDA15,OUT049,train,3735.1380
1,-1.477653,-0.907063,-1.493696,-1.334228,0,0,1,0,0,0,...,0,0,1,1,0,0,DRC01,OUT018,train,443.4228
2,1.011839,-0.956000,0.009874,-0.139681,0,1,0,0,0,0,...,0,0,1,0,1,0,FDN15,OUT049,train,2097.2700
3,1.377308,-1.281712,0.661838,-0.020226,0,0,1,0,0,0,...,0,0,1,0,1,0,FDX07,OUT010,train,732.3800
4,-0.830557,-1.281712,-1.403623,1.293777,0,1,0,0,0,0,...,0,0,1,0,0,1,NCD19,OUT013,train,994.7052
5,-0.515609,-1.281712,-1.443256,-1.334228,0,0,1,0,0,1,...,0,0,1,0,1,0,FDP36,OUT018,train,556.6088
6,0.184158,-1.034104,-1.342458,1.293777,0,0,1,0,0,0,...,0,0,1,0,1,0,FDO10,OUT013,train,343.5528
7,1.334312,1.195510,-0.535442,1.532686,0,1,0,0,0,0,...,1,0,1,0,1,0,FDP10,OUT027,train,4022.7636
8,0.732362,-0.957418,-0.709230,-0.498045,0,0,1,0,0,0,...,0,0,1,0,1,0,FDH17,OUT045,train,1076.5986
9,1.377308,0.553801,0.754073,-1.095319,0,0,1,0,0,0,...,0,0,1,0,1,0,FDU28,OUT017,train,4710.5350


In [35]:
train_preprocessed = data_preprocessed[data_preprocessed['source']=='train'].drop(columns='source')
test_preprocessed = data_preprocessed[data_preprocessed['source']=='test'].drop(columns=['source', 'Item_Outlet_Sales'])

In [36]:
train_preprocessed

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Fat_Content_LF,Item_Fat_Content_Low Fat,Item_Fat_Content_Regular,Item_Fat_Content_low fat,Item_Fat_Content_reg,Item_Type_Baking Goods,...,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3,source_test,source_train,Item_Identifier_code_DR,Item_Identifier_code_FD,Item_Identifier_code_NC,Item_Identifier,Outlet_Identifier,Item_Outlet_Sales
0,-0.751014,-0.969852,1.752511,-0.139681,0,1,0,0,0,0,...,0,0,0,1,0,1,0,FDA15,OUT049,3735.1380
1,-1.477653,-0.907063,-1.493696,-1.334228,0,0,1,0,0,0,...,1,0,0,1,1,0,0,DRC01,OUT018,443.4228
2,1.011839,-0.956000,0.009874,-0.139681,0,1,0,0,0,0,...,0,0,0,1,0,1,0,FDN15,OUT049,2097.2700
3,1.377308,-1.281712,0.661838,-0.020226,0,0,1,0,0,0,...,0,0,0,1,0,1,0,FDX07,OUT010,732.3800
4,-0.830557,-1.281712,-1.403623,1.293777,0,1,0,0,0,0,...,0,0,0,1,0,0,1,NCD19,OUT013,994.7052
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8518,-1.274495,-0.178196,1.184137,1.293777,0,1,0,0,0,0,...,0,0,0,1,0,1,0,FDF22,OUT013,2778.3834
8519,-0.948797,-0.368666,-0.529083,-0.498045,0,0,1,0,0,1,...,0,0,0,1,0,1,0,FDS36,OUT045,549.2850
8520,-0.471537,-0.597910,-0.900101,-0.736955,0,1,0,0,0,0,...,0,0,0,1,0,0,1,NCJ29,OUT035,1193.1136
8521,-1.200326,1.540475,-0.610001,-1.334228,0,0,1,0,0,0,...,1,0,0,1,0,1,0,FDN46,OUT018,1845.5976


In [37]:
test_preprocessed

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Fat_Content_LF,Item_Fat_Content_Low Fat,Item_Fat_Content_Regular,Item_Fat_Content_low fat,Item_Fat_Content_reg,Item_Type_Baking Goods,...,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3,source_test,source_train,Item_Identifier_code_DR,Item_Identifier_code_FD,Item_Identifier_code_NC,Item_Identifier,Outlet_Identifier
8523,1.710530,-1.134699,-0.533831,-0.139681,0,1,0,0,0,0,...,1,0,0,1,0,0,1,0,FDW58,OUT049
8524,-0.965996,-0.534917,-0.864708,-1.095319,0,0,0,0,1,0,...,1,0,0,1,0,0,1,0,FDW14,OUT017
8525,0.388391,0.653405,1.622763,-0.020226,0,1,0,0,0,0,...,0,0,0,1,0,0,0,1,NCN55,OUT010
8526,-1.177753,-0.982657,0.225966,-1.095319,0,1,0,0,0,0,...,1,0,0,1,0,0,1,0,FDQ58,OUT017
8527,0.173409,1.023121,1.501577,1.532686,0,0,1,0,0,0,...,0,0,1,1,0,0,1,0,FDY38,OUT027
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14199,-0.493036,-1.019425,0.005000,0.099229,0,0,1,0,0,0,...,1,0,0,1,0,0,1,0,FDB58,OUT046
14200,-1.116483,1.497142,0.453249,-1.334228,0,0,1,0,0,0,...,0,1,0,1,0,0,1,0,FDD47,OUT018
14201,-0.600527,0.147226,-0.358558,-0.498045,0,1,0,0,0,0,...,1,0,0,1,0,0,0,1,NCO17,OUT045
14202,0.538878,-1.281712,1.185747,-1.095319,0,0,1,0,0,0,...,1,0,0,1,0,0,1,0,FDJ26,OUT017


In [38]:
train_preprocessed.to_csv('train_preprocesssed.csv', index = False)
test_preprocessed.to_csv("test_preprocessed.csv", index = False)

In [39]:
x = train_preprocessed.drop(columns = ["Item_Outlet_Sales", "Item_Identifier", "Outlet_Identifier"])
y = train_preprocessed['Item_Outlet_Sales']

In [40]:
x.head()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Fat_Content_LF,Item_Fat_Content_Low Fat,Item_Fat_Content_Regular,Item_Fat_Content_low fat,Item_Fat_Content_reg,Item_Type_Baking Goods,...,Outlet_Location_Type_Tier 3,Outlet_Type_Grocery Store,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3,source_test,source_train,Item_Identifier_code_DR,Item_Identifier_code_FD,Item_Identifier_code_NC
0,-0.751014,-0.969852,1.752511,-0.139681,0,1,0,0,0,0,...,0,0,1,0,0,0,1,0,1,0
1,-1.477653,-0.907063,-1.493696,-1.334228,0,0,1,0,0,0,...,1,0,0,1,0,0,1,1,0,0
2,1.011839,-0.956000,0.009874,-0.139681,0,1,0,0,0,0,...,0,0,1,0,0,0,1,0,1,0
3,1.377308,-1.281712,0.661838,-0.020226,0,0,1,0,0,0,...,1,1,0,0,0,0,1,0,1,0
4,-0.830557,-1.281712,-1.403623,1.293777,0,1,0,0,0,0,...,1,0,1,0,0,0,1,0,0,1


In [41]:
y.head()

0    3735.1380
1     443.4228
2    2097.2700
3     732.3800
4     994.7052
Name: Item_Outlet_Sales, dtype: float64

In [42]:
train_x, test_x, train_y, test_y = train_test_split(x,y, test_size = 0.2, random_state=32)

In [43]:
model = LinearRegression()
model.fit(train_x, train_y)

LinearRegression()

In [44]:
model.coef_

array([   5.81371878,   -8.93036689,  978.30649403,  104.51229261,
        -44.94697787,   -4.96280519,   38.30731431,   24.70561569,
        -13.10314694,   -1.51710069,    5.95657   ,  -12.55431396,
         32.28804788,  -72.43164637,  -35.62293663,   45.24361354,
       -103.13046741,  -24.10120945,   -3.72611379,   23.37924889,
          2.04308158,  175.37882075,   22.49112952, -118.04450202,
         64.34777814, -448.7613984 ,  -85.21703104,  153.22118561,
        -96.31944818, -433.29944593,  605.16929836,  168.58538272,
        -94.65808221,  380.81324561, -149.53370654,  -85.21703104,
        359.31614364, -274.0991126 , -202.01990686,  227.14848612,
        -25.12857926, -882.06084433,  373.21099415,  -96.31944818,
        605.16929836,    0.        ,    0.        ,   62.96920925,
        -37.18496759,  -25.78424165])

In [45]:
model.intercept_

2018.8618414802581

In [46]:
train_X_pred = model.predict(train_x)
test_X_pred = model.predict(test_x)

train_X_RMSE = np.sqrt(mean_squared_error(train_y, train_X_pred))
test_X_RMSE = np.sqrt(mean_squared_error(test_y, test_X_pred))

In [47]:
print("Train RMSE : ",train_X_RMSE)
print("Test RMSE : ",test_X_RMSE)

Train RMSE :  1124.9617670679029
Test RMSE :  1136.7304618857552


In [48]:
train_X_R2 = r2_score(train_y, train_X_pred)
test_X_R2 = r2_score(test_y, test_X_pred)

In [49]:
print("Train R2 : ",train_X_R2)
print("Test R2 : ",test_X_R2)

Train R2 :  0.564601725089632
Test R2 :  0.5592405866432242


In [65]:
train_preprocessed.head()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Fat_Content_LF,Item_Fat_Content_Low Fat,Item_Fat_Content_Regular,Item_Fat_Content_low fat,Item_Fat_Content_reg,Item_Type_Baking Goods,...,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3,source_test,source_train,Item_Identifier_code_DR,Item_Identifier_code_FD,Item_Identifier_code_NC,Item_Identifier,Outlet_Identifier,Item_Outlet_Sales
0,-0.751014,-0.969852,1.752511,-0.139681,0,1,0,0,0,0,...,0,0,0,1,0,1,0,FDA15,OUT049,3735.1380
1,-1.477653,-0.907063,-1.493696,-1.334228,0,0,1,0,0,0,...,1,0,0,1,1,0,0,DRC01,OUT018,443.4228
2,1.011839,-0.956000,0.009874,-0.139681,0,1,0,0,0,0,...,0,0,0,1,0,1,0,FDN15,OUT049,2097.2700
3,1.377308,-1.281712,0.661838,-0.020226,0,0,1,0,0,0,...,0,0,0,1,0,1,0,FDX07,OUT010,732.3800
4,-0.830557,-1.281712,-1.403623,1.293777,0,1,0,0,0,0,...,0,0,0,1,0,0,1,NCD19,OUT013,994.7052


In [71]:
test.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,source
0,FDW58,20.750,Low Fat,0.007565,Snack Foods,107.8622,OUT049,1999,Medium,Tier 1,Supermarket Type1,test
1,FDW14,8.300,reg,0.038428,Dairy,87.3198,OUT017,2007,NaN,Tier 2,Supermarket Type1,test
2,NCN55,14.600,Low Fat,0.099575,Others,241.7538,OUT010,1998,NaN,Tier 3,Grocery Store,test
3,FDQ58,7.315,Low Fat,0.015388,Snack Foods,155.0340,OUT017,2007,NaN,Tier 2,Supermarket Type1,test
4,FDY38,NaN,Regular,0.118599,Dairy,234.2300,OUT027,1985,Medium,Tier 3,Supermarket Type3,test


In [73]:
test_preprocessed.head()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Fat_Content_LF,Item_Fat_Content_Low Fat,Item_Fat_Content_Regular,Item_Fat_Content_low fat,Item_Fat_Content_reg,Item_Type_Baking Goods,...,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3,source_test,source_train,Item_Identifier_code_DR,Item_Identifier_code_FD,Item_Identifier_code_NC,Item_Identifier,Outlet_Identifier
8523,1.710530,-1.134699,-0.533831,-0.139681,0,1,0,0,0,0,...,1,0,0,1,0,0,1,0,FDW58,OUT049
8524,-0.965996,-0.534917,-0.864708,-1.095319,0,0,0,0,1,0,...,1,0,0,1,0,0,1,0,FDW14,OUT017
8525,0.388391,0.653405,1.622763,-0.020226,0,1,0,0,0,0,...,0,0,0,1,0,0,0,1,NCN55,OUT010
8526,-1.177753,-0.982657,0.225966,-1.095319,0,1,0,0,0,0,...,1,0,0,1,0,0,1,0,FDQ58,OUT017
8527,0.173409,1.023121,1.501577,1.532686,0,0,1,0,0,0,...,0,0,1,1,0,0,1,0,FDY38,OUT027


In [75]:
test_pred = model.predict(test_preprocessed.drop(columns = ["Item_Identifier", "Outlet_Identifier"]))

In [76]:
test_pred_modified = np.where(test_pred<0,0,test_pred)

In [77]:
test['Item_Outlet_Sales'] = test_pred_modified

In [78]:
Output = test[[ "Item_Identifier", "Outlet_Identifier","Item_Outlet_Sales"]]

In [79]:
Output.to_csv("Result_Linear_Regression_with_80percent_traindata_abs.csv", index = False)